In [1]:
import pandas as pd
import requests
from Bio import SeqIO

In [2]:
lineages = ['BQ.1.1']
thr = 0.9
out_path = '/media/NASII/Datos/ANALYSIS/MISC/covid_analysis/variant_fastas/'

In [3]:
def get_markers(lineage, thr):
    # Get SNPs above threshold from Lapis and save to df
    url = f'https://lapis.cov-spectrum.org/open/v1/sample/nuc-mutations?pangoLineage={lineage}&downloadAsFile=false&dataFormat=json'
    print(f"Url is {url}")
    r = requests.get(url)
    result = r.json()
    df = pd.json_normalize(result['data'])
    df.drop(df[df['proportion'] < thr].index, inplace=True)
    df.drop(columns=['proportion', 'count'], inplace=True)

    # Get REF, POS and ALT from df 
    df[['REF', 'POS', 'ALT']] = df['mutation'].str.extract('(\D+)(\d+)(\D+)', expand=True)

    return df

In [4]:
def consensus(df):
    # Read Wuhan reference
    wuhan_seq = SeqIO.parse(open('/home/laura/iisgm/NC_045512.2.fasta'),'fasta')
    # Extract fasta sequence
    for fasta in wuhan_seq:
        seq = list(str(fasta.seq))
    # Modify reference to create lineage sequence
    for _, row in df.iterrows():
        pos = int(row['POS']) - 1
        alt = row['ALT']
        seq[pos] = alt

    return ''.join(seq)

In [5]:
for lineage in lineages:
    df = get_markers(lineage, thr)
    variant_fasta = consensus(df)
    # Outuput SNP tablo to new_file
    df.to_csv(out_path + lineage + '.csv', index=False)
    # Output sequence to new file 
    with open(out_path + lineage + '.fasta', 'w') as of:
        ident = '>' + lineage + '\n'
        of.write(ident)
        of.write(variant_fasta)

Url is https://lapis.cov-spectrum.org/open/v1/sample/nuc-mutations?pangoLineage=BQ.1.1&downloadAsFile=false&dataFormat=json
